<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/Practice_individual_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=060a04e1c84cf3ee83ab6ac9ffb36ea94c20dc42d1e5377850745a1987c4961f
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, to_timestamp, col
from datetime import datetime
from pyspark.sql.types import DateType

# Самостоятельное задание  №1. Погода

In [87]:
spark = SparkSession.builder \
.appName("Weather") \
.getOrCreate()

df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/weather_data.csv", header=True)
print(df.dtypes)
df = df.withColumn("date", to_date(col("date"), 'yyyy-MM-dd'))
print(df.dtypes)


df = df.withColumn("temperature", col("temperature").cast("float"))
df = df.withColumn("precipitation", col("precipitation").cast("float"))
df = df.withColumn("wind_speed", col("wind_speed").cast("float"))
# вот тут как раз видно отложенное выполнение("ленивое выполнение") спарком на практике
# тут типы данных не преобразовались, но ниже в sql запросах теперь все корректно



print(df.count())
df.show()


[('station_id', 'string'), ('date', 'string'), ('temperature', 'string'), ('precipitation', 'string'), ('wind_speed', 'string')]
[('station_id', 'string'), ('date', 'date'), ('temperature', 'string'), ('precipitation', 'string'), ('wind_speed', 'string')]
1000
+----------+----------+-----------+-------------+----------+
|station_id|      date|temperature|precipitation|wind_speed|
+----------+----------+-----------+-------------+----------+
| station_7|2022-06-28|  -6.751842|    23.670044|     5.459|
| station_4|2020-04-07|  -9.574844|    2.9858243| 6.8285055|
| station_8|2018-12-22|   19.34342|    33.582115|  8.975576|
| station_5|2021-09-09|  30.880953|    29.110437| 18.264654|
| station_7|2023-02-07|  23.459047|    49.510445| 3.7878683|
|station_10|2018-05-07|  27.525618|    11.957939|0.10053105|
| station_3|2019-08-27|  4.2879057|    26.536867| 1.2019283|
| station_7|2023-04-13|  19.244547|    32.547867| 18.095327|
| station_8|2020-10-10|  1.2541745|    32.229557| 12.950329|
| stati

In [88]:
for column in df.columns:
  print(df[column].isNull())

Column<'(station_id IS NULL)'>
Column<'(date IS NULL)'>
Column<'(temperature IS NULL)'>
Column<'(precipitation IS NULL)'>
Column<'(wind_speed IS NULL)'>


In [89]:
# пропущенных значений нет

- Найдите топ-5 самых жарких дней за все время наблюдений.
- Найдите метеостанцию с наибольшим количеством осадков за последний год.
- Подсчитайте среднюю температуру по месяцам за все время наблюдений.



In [90]:
df.createOrReplaceTempView("df")

top_hot = spark.sql("""
SELECT date, temperature
FROM df
ORDER BY df.temperature DESC
LIMIT 5;
""")
top_hot.show()

+----------+-----------+
|      date|temperature|
+----------+-----------+
|2021-08-20|   39.98283|
|2023-12-02|  39.967976|
|2022-03-28|  39.824688|
|2019-02-11|  39.767376|
|2020-06-10|   39.69148|
+----------+-----------+



In [91]:
top_precipitation = spark.sql("""
SELECT station_id, sum(precipitation)
FROM df
WHERE date > '2023-09-24'
GROUP BY station_id
ORDER BY sum(precipitation) DESC
LIMIT 1;
""")
top_precipitation.show()

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5|164.60128784179688|
+----------+------------------+



In [92]:
from pyspark.sql.functions import year, month, avg

In [93]:
df_monthly = df.groupBy(month("date").alias("month")).agg(
    avg("temperature").alias("average_temperature"),
)
df_sorted = df_monthly.orderBy(col("average_temperature").desc()) # использование col помогает во многом

df_sorted.show()

+-----+-------------------+
|month|average_temperature|
+-----+-------------------+
|    6| 13.421092370936744|
|    4| 12.024529053670603|
|    1| 11.356518470007797|
|   12| 11.218592057529005|
|    8| 10.967800280712183|
|    5|  9.902883278588726|
|    9|  9.596744181906304|
|   10|  9.098843419781097|
|    2|   9.06722993850708|
|   11|  7.265890174138714|
|    3|  7.244080115937525|
|    7|  6.185718382110021|
+-----+-------------------+



In [94]:
spark.stop()

# Самостоятельное задание  №2. Книги и авторы

In [95]:
from pyspark.sql.types import IntegerType, StringType, DateType, FloatType

In [96]:
spark = SparkSession.builder \
.appName("Books") \
.getOrCreate()

books_df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/books.csv", header=True)
authors_df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/authors.csv", header=True)
books_df.show()
authors_df.show()
print(books_df.dtypes)
print(authors_df.dtypes)
book_data_types = {
    "book_id": IntegerType(),
    "title": StringType(),
    "author_id": IntegerType(),
    "genre": StringType(),
    "price": FloatType(),
    "publish_date": DateType()
}
authors_data_types = {
    "author_id": IntegerType(),
    "name": StringType(),
    "birth_date": DateType(),
    "country": StringType()
}
for column_name, data_type in book_data_types.items():
  books_df = books_df.withColumn(column_name, col(column_name).cast(data_type))
for column_name, data_type in authors_data_types.items():
  authors_df = authors_df.withColumn(column_name, col(column_name).cast(data_type))

print(books_df.dtypes)
print(authors_df.dtypes)

+-------+-------+---------+-----------+-----+------------+
|book_id|  title|author_id|      genre|price|publish_date|
+-------+-------+---------+-----------+-----+------------+
|      1| Book_1|        2|    Mystery|73.57|  1980-12-31|
|      2| Book_2|        1|Non-Fiction| 41.1|  1982-12-31|
|      3| Book_3|       10|    Fiction|10.63|  1984-12-31|
|      4| Book_4|        9|Non-Fiction|46.31|  1986-12-31|
|      5| Book_5|        7|    Science|31.13|  1988-12-31|
|      6| Book_6|        4|Non-Fiction| 83.7|  1990-12-31|
|      7| Book_7|        6|Non-Fiction|40.36|  1992-12-31|
|      8| Book_8|        2|Non-Fiction|84.48|  1994-12-31|
|      9| Book_9|        7|    Fantasy|10.05|  1996-12-31|
|     10|Book_10|        2|    Science| 37.7|  1998-12-31|
|     11|Book_11|       10|Non-Fiction| 31.7|  2000-12-31|
|     12|Book_12|        8|Non-Fiction|31.02|  2002-12-31|
|     13|Book_13|        8|Non-Fiction|16.14|  2004-12-31|
|     14|Book_14|        1|    Fiction|26.84|  2006-12-3

In [97]:
df_join = books_df.join(authors_df, "author_id", "left")
df_join.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|
|        2|      8| Book_8|Non-Fiction|84.48|  1994-12-31| Author_2|1965-12-31|   Canada|
|        7

- Найдите топ-5 авторов, книги которых принесли наибольшую выручку.
- Найдите количество книг в каждом жанре.
- Подсчитайте среднюю цену книг по каждому автору.
- Найдите книги, опубликованные после 2000 года, и отсортируйте их по цене.

In [98]:
from pyspark.sql.functions import sum, max, min

In [99]:
grouped_df = df_join.groupBy("author_id").agg(sum("price").alias("sum_price"))
grouped_df = grouped_df.orderBy(col("sum_price").desc()).limit(5)
grouped_df.show()

+---------+------------------+
|author_id|         sum_price|
+---------+------------------+
|        2| 231.9700050354004|
|        7|132.66000270843506|
|        1| 111.8599967956543|
|        8|107.15999984741211|
|        5| 88.83000183105469|
+---------+------------------+



In [100]:
df_join.groupBy("genre").count().orderBy(col("count").desc()).show()

+-----------+-----+
|      genre|count|
+-----------+-----+
|Non-Fiction|    9|
|    Science|    3|
|    Fiction|    3|
|    Fantasy|    3|
|    Mystery|    2|
+-----------+-----+



In [101]:
df_join.groupBy("author_id").agg(avg("price").alias("avg_price")).orderBy(col("avg_price").desc()).show()

+---------+------------------+
|author_id|         avg_price|
+---------+------------------+
|        5| 88.83000183105469|
|        4| 83.69999694824219|
|        2|  57.9925012588501|
|        9|46.310001373291016|
|        7|44.220000902811684|
|        6| 43.96500015258789|
|        1| 37.28666559855143|
|        8| 35.71999994913737|
|       10|21.165000438690186|
+---------+------------------+



In [102]:
df_join.where(col("publish_date") >= "2000-01-01").orderBy(col("price").desc()).show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        7|     20|Book_20|    Mystery|91.48|  2018-12-31| Author_7|1990-12-31|      USA|
|        5|     19|Book_19|    Science|88.83|  2016-12-31| Author_5|1980-12-31|      USA|
|        8|     15|Book_15|    Fantasy| 60.0|  2008-12-31| Author_8|1995-12-31|Australia|
|        6|     17|Book_17|    Fantasy|47.57|  2012-12-31| Author_6|1985-12-31|      USA|
|        1|     18|Book_18|Non-Fiction|43.92|  2014-12-31| Author_1|1960-12-31|    India|
|        2|     16|Book_16|    Fiction|36.22|  2010-12-31| Author_2|1965-12-31|   Canada|
|       10|     11|Book_11|Non-Fiction| 31.7|  2000-12-31|Author_10|2005-12-31|    India|
|        8|     12|Book_12|Non-Fiction|31.02|  2002-12-31| Author_8|1995-12-31|Australia|
|        1

In [103]:
spark.stop()

# Самостоятельное задание  №3. Фильмы и актеры. С использованием временных таблиц и SQL


In [107]:
spark = SparkSession.builder \
.appName("Books") \
.getOrCreate()

movies_df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/movies.csv", header=True)
movie_actors_df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/movie_actors.csv", header=True)
actors_df = spark.read.csv("/content/drive/MyDrive/dataenj/pySpark_practice/actors.csv", header=True)
movies_df.show()
movie_actors_df.show()
actors_df.show()
print(movies_df.dtypes)
print(actors_df.dtypes)
print(movie_actors_df.dtypes)

movies_data_types = {
    "movie_id": IntegerType(),
    "budget": FloatType(),
    "release_date": DateType()
}

movie_actors_data_types = {
    "movie_id": IntegerType(),
    "actor_id": IntegerType()
}
actors_data_types = {
    "actor_id": IntegerType(),
    "birth_date": DateType()
}

for column_name, data_type in movies_data_types.items():
  movies_df = movies_df.withColumn(column_name, col(column_name).cast(data_type))
for column_name, data_type in movie_actors_data_types.items():
  movie_actors_df = movie_actors_df.withColumn(column_name, col(column_name).cast(data_type))
for column_name, data_type in actors_data_types.items():
  actors_df = actors_df.withColumn(column_name, col(column_name).cast(data_type))

print(movies_df.dtypes)
print(actors_df.dtypes)
print(movie_actors_df.dtypes)


+--------+--------+------+------------+-----------+
|movie_id|   title| genre|release_date|     budget|
+--------+--------+------+------------+-----------+
|       1| Movie_1|Horror|  2000-12-31|86600583.11|
|       2| Movie_2|Comedy|  2001-12-31|12747400.83|
|       3| Movie_3|Action|  2002-12-31| 18015774.7|
|       4| Movie_4| Drama|  2003-12-31|48176120.61|
|       5| Movie_5| Drama|  2004-12-31| 74050161.1|
|       6| Movie_6|Action|  2005-12-31|14761218.31|
|       7| Movie_7| Drama|  2006-12-31|44567036.43|
|       8| Movie_8| Drama|  2007-12-31|48802276.17|
|       9| Movie_9|Action|  2008-12-31|22016278.53|
|      10|Movie_10|Action|  2009-12-31|12440279.29|
|      11|Movie_11|Comedy|  2010-12-31|83805671.38|
|      12|Movie_12|Comedy|  2011-12-31|50744099.33|
|      13|Movie_13|Action|  2012-12-31| 2423742.36|
|      14|Movie_14|Sci-Fi|  2013-12-31|80495148.83|
|      15|Movie_15| Drama|  2014-12-31|98098586.74|
|      16|Movie_16|Comedy|  2015-12-31|60986693.35|
|      17|Mo

In [108]:
movies_df.createOrReplaceTempView("movies")
movie_actors_df.createOrReplaceTempView("movie_actors")
actors_df.createOrReplaceTempView("actors")

- Найдите топ-5 жанров по количеству фильмов.
- Найдите актера с наибольшим количеством фильмов.
- Подсчитайте средний бюджет фильмов по жанрам.
- Найдите фильмы, в которых снялось более одного актера из одной страны.

In [110]:
top_genres = spark.sql("""
SELECT m.genre, count(movie_id) AS total_films
FROM movies m
GROUP BY m.genre
ORDER BY total_films DESC
LIMIT 5;
""")
top_genres.show()

+------+-----------+
| genre|total_films|
+------+-----------+
| Drama|          6|
|Action|          6|
|Comedy|          4|
|Horror|          2|
|Sci-Fi|          2|
+------+-----------+



In [138]:
top_actor = spark.sql("""
SELECT a.name, count(DISTINCT(ma.movie_id)) AS num_films
FROM actors a
INNER JOIN movie_actors ma
ON ma.actor_id = a.actor_id
GROUP BY a.name
ORDER BY num_films DESC
LIMIT 1;

""")
top_actor.show()

+--------+---------+
|    name|num_films|
+--------+---------+
|Actor_24|        5|
+--------+---------+



In [140]:
avg_budget = spark.sql("""
SELECT genre, avg(budget) AS avg_budget
FROM movies m
GROUP BY m.genre
ORDER BY avg_budget DESC;
""")
avg_budget.show()

+------+--------------+
| genre|    avg_budget|
+------+--------------+
|Horror|   8.7281876E7|
|Sci-Fi|   7.8097152E7|
| Drama|   6.0760218E7|
|Comedy| 5.207096625E7|
|Action|2.7492741875E7|
+------+--------------+



In [141]:
two_or_more_from_same_country = spark.sql("""
SELECT movie_id
FROM movie_actors ma
INNER JOIN actors a
ON ma.actor_id = a.actor_id
GROUP BY ma.movie_id
HAVING count(DISTINCT(ma.actor_id)) > count(DISTINCT(a.country))
""")## если различных актеров больше чем различных стран, значит 2 или больше из одной страны.
two_or_more_from_same_country.show()

+--------+
|movie_id|
+--------+
|       1|
|       3|
|      15|
|       7|
|      10|
|      18|
+--------+

